This file contains python code to 
test old model code vs new model code (integration for once)

In [1]:
import sys
sys.path.append("../../mypkg")

In [2]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns

from easydict import EasyDict as edict
from tqdm import trange, tqdm
from pprint import pprint
import itertools
from scipy.stats import chi2


In [3]:
# This will reload all imports as soon as the code changes
%load_ext autoreload
%autoreload 2

In [4]:
from constants import DATA_ROOT, RES_ROOT, FIG_ROOT, MIDRES_ROOT
from hdf_utils.data_gen import gen_simu_sinica_dataset
from hdf_utils.SIS import SIS_GLIM
from hdf_utils.utils import gen_int_ws, integration_fn
from utils.matrix import col_vec_fn, col_vec2mat_fn, conju_grad, svd_inverse, cholesky_inv
from utils.functions import logit_fn
from utils.misc import save_pkl, load_pkl
from splines import obt_bsp_obasis_Rfn, obt_bsp_basis_Rfn_wrapper
from projection import euclidean_proj_l1ball
from optimization.opt import HDFOpt
from scenarios.simu_linear_sinica import settings
from hdf_utils.fns_sinica import  fourier_basis_fn
from models.linear_model import LinearModel
from models1.linear_model import LinearModel as LinearModel1
from models.logistic_model import LogisticModel
from models1.logistic_model import LogisticModel as LogisticModel1

from joblib import Parallel, delayed

In [5]:
plt.style.use(FIG_ROOT/"base.mplstyle")
torch.set_default_tensor_type(torch.DoubleTensor)

# Params

In [6]:
np.random.seed(0)
c = 0.0

setting = settings.cmpns1
data_gen_params = setting.data_gen_params
data_gen_params.cs = data_gen_params.cs_fn(c)
data_gen_params.gt_beta = data_gen_params.beta_fn(data_gen_params.cs)
data_gen_params.gt_alp = [1, 2]
data_gen_params.types_ = ["int", "c"]
data_gen_params.q = 2
#data_gen_params.gt_beta[:, -1] = 0
x = np.linspace(0, 1, data_gen_params.npts)

num_rep = 200
Cmat = np.eye(data_gen_params.d - len(setting.sel_idx))
save_dir = RES_ROOT/f"simu_setting{setting.setting}_test"
if not save_dir.exists():
    save_dir.mkdir()
Cmat

array([[1.]])

In [7]:
cur_data = gen_simu_sinica_dataset(n=data_gen_params.n, 
                                   d=data_gen_params.d, 
                                   q=data_gen_params.q, 
                                   types_=data_gen_params.types_, 
                                   gt_alp=data_gen_params.gt_alp, 
                                   gt_beta=data_gen_params.gt_beta, 
                                   x=x,
                                   data_type=data_gen_params.data_type,
                                   data_params=data_gen_params.data_params, 
                                   seed=0, 
                                   verbose=2);

In [8]:
N = 10
basis_mat = torch.tensor(obt_bsp_obasis_Rfn(x, N=N, bsp_ord=4));

R[write to console]: 
Attaching package: ‘orthogonalsplinebasis’


R[write to console]: The following object is masked from ‘package:stats’:

    integrate




In [9]:
# test linear model
model = LinearModel(
    Y = cur_data.Y,
    X = cur_data.X, 
    Z = cur_data.Z, 
    ws= None, 
    basis_mat=basis_mat
)
model1 = LinearModel1(
    Y = cur_data.Y,
    X = cur_data.X, 
    Z = cur_data.Z, 
    ws= None, 
    basis_mat=basis_mat
)

alp = torch.randn(2)
Gam = torch.randn(N, data_gen_params.d);
model._linear_term_der()
model1._linear_term_der()
diffs = [
    torch.norm(model.lin_tm_der - model1.lin_tm_der),
    torch.norm(model._obt_lin_tm(alp, Gam)- model1._obt_lin_tm(alp, Gam)), 
    torch.norm(model.log_lik(alp, Gam)- model1.log_lik(alp, Gam)), 
    torch.norm(model.log_lik_der1(alp, Gam)- model1.log_lik_der1(alp, Gam)), 
    torch.norm(model.log_lik_der2(alp, Gam)- model1.log_lik_der2(alp, Gam)), 
]

[tensor(0.), tensor(7.4546e-14), tensor(0.), tensor(5.4386e-14), tensor(0.)]

In [10]:
# test logi model
model = LogisticModel(
    Y = (cur_data.Y>1).float(),
    X = cur_data.X, 
    Z = cur_data.Z, 
    ws= None, 
    basis_mat=basis_mat
)
model1 = LogisticModel1(
    Y = (cur_data.Y>1).float(),
    X = cur_data.X, 
    Z = cur_data.Z, 
    ws= None, 
    basis_mat=basis_mat
)

alp = torch.randn(2)
Gam = torch.randn(N, data_gen_params.d);
model._linear_term_der()
model1._linear_term_der()
diffs = [
    torch.norm(model.lin_tm_der - model1.lin_tm_der),
    torch.norm(model._obt_lin_tm(alp, Gam)- model1._obt_lin_tm(alp, Gam)), 
    torch.norm(model.log_lik(alp, Gam)- model1.log_lik(alp, Gam)), 
    torch.norm(model.log_lik_der1(alp, Gam)- model1.log_lik_der1(alp, Gam)), 
    torch.norm(model.log_lik_der2(alp, Gam)- model1.log_lik_der2(alp, Gam)), 
]

[tensor(0.),
 tensor(7.0080e-14),
 tensor(0.),
 tensor(3.0120e-15),
 tensor(8.6893e-14)]